### Code réalisé par l'équipe 8.
### Équipe : Anas Bouhemhem 1953369
###          Bouh Abdillahi 1940646

Laboratoire 1 

In [51]:
import numpy as np
import pandas as pd 
import yaml
from yaml.loader import SafeLoader
import random
import math
import matplotlib.pyplot as plt
import operator

import os
from pathlib import Path
def run_file(path):
    return exec(open(path).read())

run_file("test.py")
print(my_variable)


Hello!


NameError: name 'my_variable' is not defined

In [38]:
# import pyyaml module
# Open the file and load the file
with open('cas8.yaml') as f:
    data = yaml.load(f, Loader=SafeLoader)
    print(data)

{'ETUDE_IMPORTANT': {'SCENARIO': 'Uma', 'ANT_COORD_GEN': 'g', 'UE_COORD_GEN': 'a', 'DEVICES': {'Antenna1': {'number': 25}, 'UE1-App1': {'number': 30}, 'UE2-App2': {'number': 20}}, 'GEOMETRY': {'Surface': {'rectangle': {'length': 5000, 'heigth': 4000}}}}}


In [39]:
# import pyyaml module
# Open the file and load the file
with open('devices_db.yaml') as f:
    data = yaml.load(f, Loader=SafeLoader)
    print(data)

{'ANTENNAS': {'Antenna1': {'type': 'antenna', 'name': 'Antenna1', 'height': 10, 'scenario': 'UMi', 'frequency': 28, 'gain': 40}, 'Antenna2': {'type': 'antenna', 'name': 'Antenna2', 'height': 35, 'scenario': 'RMa', 'frequency': 0.9, 'gain': 45}, 'Antenna3': {'type': 'antenna', 'name': 'Antenna1', 'height': 10, 'scenario': 'UMi', 'frequency': 28, 'gain': 45}, 'Antenna4': {'type': 'antenna', 'name': 'Antenna2', 'height': 37, 'scenario': 'RMa', 'frequency': 0.9, 'gain': 45}}, 'UES': {'UE1-App1': {'type': 'ue', 'name': 'UE1-App1', 'app': 'app1', 'height': 1.5}, 'UE2-App2': {'type': 'ue', 'name': 'UE2-App2', 'app': 'app2', 'height': 1.5}}}


### Routine de génération de coordonnées de la structure en grille 

In [40]:
def fill_up_the_lattice(N, lh, lv, nh, nv):
    """Function appelée par get_rectangle_lattice_coords()"""
    
    def get_delta1d(L, n):
        return L/(n + 1)
    
    coords = []
    deltav = get_delta1d(lv, nv)
    deltah = get_delta1d(lh, nh)
    line = 1
    y = deltav
    count = 0
    while count < N:
        if count + nh < N:
            x = deltah
            for  i in range(nh):
                # Fill up the horizontal line
                coords.append((x,y))
                x = x + deltah
                count += 1
            line += 1
        else:
            deltah = get_delta1d(lh, N - count)
            x = deltah
            for i in range(N - count):
                # Fill up the last horizontal line
                coords.append((x,y))
                x = x + deltah
                count += 1
            line += 1
        y = y +deltav
    return coords



def get_rectangle_lattice_coords(lh, lv, N, Np, nh, nv):
    """Function appelee par gen_lattice_coords()"""
    
    if Np > N:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv)
    elif Np < N:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv + 1)
    else:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv)
    return coords


# In[170]:


def gen_lattice_coords(terrain_shape: dict, N: int):
    """Génère un ensemble de N coordonnées placées en grille 
       sur un terrain rectangulaire
    
       Args: terrain_shape: dictionary {'rectangle': {'length' : lh,
                                                   'height' : lv}
           lh and lv are given in the case file"""

    shape = list(terrain_shape.keys())[0]
    lh = terrain_shape[shape]['length']
    lv = terrain_shape[shape]['height']
    R = lv / lh    
    nv = round(math.sqrt(N / R))
    nh = round(R * nv)
    Np = nh * nv
    if shape.lower() == 'rectangle':
        coords = get_rectangle_lattice_coords(lh, lv, N, Np, nh, nv)
    else:
        msg = [f"\tImproper shape ({shape}) used in the\n",
                "\tgeneration of lattice coordinates.\n"
                "\tValid values: ['rectangle']"]
        ERROR(''.join(msg), 2)
    return coords

### Routine des coordonnées en grille

In [41]:
shape= {'rectangle': {'length': 200, 'height': 200}}
N = 32
coords = gen_lattice_coords(shape, N)
print(coords)

[(28.571428571428573, 28.571428571428573), (57.142857142857146, 28.571428571428573), (85.71428571428572, 28.571428571428573), (114.28571428571429, 28.571428571428573), (142.85714285714286, 28.571428571428573), (171.42857142857144, 28.571428571428573), (28.571428571428573, 57.142857142857146), (57.142857142857146, 57.142857142857146), (85.71428571428572, 57.142857142857146), (114.28571428571429, 57.142857142857146), (142.85714285714286, 57.142857142857146), (171.42857142857144, 57.142857142857146), (28.571428571428573, 85.71428571428572), (57.142857142857146, 85.71428571428572), (85.71428571428572, 85.71428571428572), (114.28571428571429, 85.71428571428572), (142.85714285714286, 85.71428571428572), (171.42857142857144, 85.71428571428572), (28.571428571428573, 114.28571428571429), (57.142857142857146, 114.28571428571429), (85.71428571428572, 114.28571428571429), (114.28571428571429, 114.28571428571429), (142.85714285714286, 114.28571428571429), (171.42857142857144, 114.28571428571429), (

### Calcul de Pathloss

In [42]:
def calcul_pathloss(distance_ue_antenne):
    """Calculate the pathloss (substitution for the 3gpp formulas)"""
    if  (distance_ue_antenne == 0) :
        pathloss=500*random.random()
    else : 
        pathloss = distance_ue_antenne
    return pathloss


### Objet Antennes et UE (User Equipement)

In [43]:

class Antenna:
    
    def __init__(self, id):
        self.id = id #id de l'antenne (int)
        self.group = None # group défini dans la base de données (str)
        self.coords = None # tuple contenant les coordonnées (x,y) 
        self.assoc_ues = [] # liste avec les id des UEs associés à l'antenne
        self.scenario = None # scénario tel que lu du fichier de cas (str)
        self.gen = None # type de géneration de coordonnées: 'g', 'a', etc. (str)
        
    
class UE:
    
    def __init__(self, id, app_name):
        self.id= id #id de l'UE (int)
        self.group = None # group défini dans la base de données (str)
        self.coords=None # tuple contenant les coordonnées (x,y) 
        self.app=app_name #nom de l'application qui tourne dans le UE (str)
        self.assoc_ant=None #id de l'antenne associée à l'UE (int)
        self.los = True # LoS ou non (bool)
        self.gen = None # type de géneration de coordonnées: 'g', 'a', etc. (str)
       

### Calcul du pathloss

In [44]:
test = calcul_pathloss(0)
print(test)

7.285608638471997


In [45]:
def place_ue_and_antennas(N_antenne, N_ue):
    shape= {'rectangle': {'length': 1000, 'height': 500}}
    N = N_antenne
    coords_antennes = gen_lattice_coords(shape, N)

    coords_antennes_x = [item[0] for item in coords_antennes]

    coords_antennes_y = [item[1] for item in coords_antennes]





    nb_ue = N_ue
    coords_ue_x = []
    coords_ue_y = []
    coords_ue = []
    index = 0

    #grid element layout generation and attribution

    while index < nb_ue:
        temp = 1000*random.random()
        coords_ue_x.append(temp)
        temp2 = 500*random.random()
        coords_ue_y.append(temp2)
        index = index + 1


    index = 0
    while index < nb_ue:
        coords_ue.append((coords_ue_x[index], coords_ue_y[index]))
        index = index + 1

    #calcul de distance 
    distance = []
    return coords_ue, coords_antennes

        




In [46]:
def calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne):
    nb_ue = N_ue 
    index_antennes = 0
    index_ue = 0
    smallestDistance = [0] * nb_ue
    N = N_antenne
    while index_ue < nb_ue:
        index_antennes = 0
        path_loss_min = 10000000000 #valeur arbitraire, il faut juste qu'elle soit assez grande pour prendre la valeur la plus petite.
        while index_antennes < N:
            temp = tuple(map(operator.sub, coords_ue[index_ue], coords_antennes[index_antennes]))
            
            if pathloss_computation == "random" :
                check = calcul_pathloss(0) #random pathloss
            if pathloss_computation == "distance" :
                check = calcul_pathloss(math.sqrt(math.pow(temp[0],2) + math.pow(temp[1],2))) #pathloss from the distance
            #check if pathloss is the smallest one in the network of antennas
            if check < path_loss_min:
                path_loss_min = check
                smallestDistance[index_ue] = (check,index_antennes + 1) #antenne commence a 1, et non a 0.

            index_antennes = index_antennes + 1
            
        index_ue = index_ue + 1



    numerant = []
    pathloss = []
    id_ue = []
    temp = 1
    for i in smallestDistance:
        numerant.append(i[1])
        pathloss.append(i[0])
        id_ue.append(temp)
        temp = temp + 1
    return id_ue, numerant, pathloss



def inv_calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne):
    nb_ue = N_ue 
    index_antennes = 0
    index_ue = 0
    N = N_antenne
    smallestDistance = [0] * N
    while index_antennes < N: 
        index_ue = 0
        path_loss_min = 10000000000 #valeur arbitraire, il faut juste qu'elle soit assez grande pour prendre la valeur la plus petite.
        while index_ue < nb_ue:
            temp = tuple(map(operator.sub, coords_ue[index_ue], coords_antennes[index_antennes]))
            
            if pathloss_computation == "random" :
                check = calcul_pathloss(0) #random pathloss
            if pathloss_computation == "distance" :
                check = calcul_pathloss(math.sqrt(math.pow(temp[0],2) + math.pow(temp[1],2))) #pathloss from the distance
            #check if pathloss is the smallest one in the network of antennas
            if check < path_loss_min:
                path_loss_min = check
                smallestDistance[index_antennes] = (check,index_ue + 1) #ue commence a 1, et non a 0.

            index_ue = index_ue + 1
            
        index_antennes = index_antennes + 1


    numerue = []
    pathloss = []
    id_ant = []
    temp = 1
    for i in smallestDistance:
        numerue.append(i[1])
        pathloss.append(i[0])
        id_ant.append(temp)
        temp = temp + 1

    return id_ant, numerue, pathloss

### Écriture dans un fichier

In [47]:
def write_file_assoc(pathloss_computation, pl_lab1_rand_eqn, assoc_lab1_rand_eqn, coords_ue, coords_antennes, N_ue, N_antenne):

    id_ue, numerant, pathloss = calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne)


    data = {"ID UE": id_ue, "ID antenne": numerant,
    "Pathloss entre UE et antenne": pathloss
    }

    id_ant, numerue, pathloss = inv_calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne)
    data_inv = {"ID antenne": id_ant, "ID UE": numerue, 
    "Pathloss entre UE et antenne": pathloss
    }

    df = pd.DataFrame(data=data)
    df_inv = pd.DataFrame(data=data_inv)

    grp = df.sort_values(by=['ID antenne'])
    new_df = grp['ID antenne'].tolist()
    new_df_id_ue = grp['ID UE'].tolist()
    list_antenne_sorted = list(range(N_antenne + 1))
    list_antenne_sorted = list_antenne_sorted[1:]
    buffer = []
    vector = [0]*N_antenne
    current_ue_selection = ""

    #This let's us associate an antenna to a list of specified UEs
    temp = 0
    for j in  id_ue :
        if temp == 0 and j == 0 :
            current_ue_selection = current_ue_selection + str(new_df_id_ue[j-1] ) + " "
        if j < 300 and j > 0 :
            if new_df[j-1] == new_df[j] and j < 300 and j > 0 :
                current_ue_selection = current_ue_selection + str(new_df_id_ue[j-1]) + " "
            else :
                buffer.append(current_ue_selection)
                vector[temp] = buffer 
                buffer = []
                current_ue_selection = ""
                temp = temp + 1

        if temp == (N_antenne-1) :
            if (j == N_ue):
                current_ue_selection = current_ue_selection + " " + str(new_df_id_ue[j-1])
                buffer.append(current_ue_selection)
                vector[temp] = buffer 

    #Dataframe to encapsulate the association antenna to list of UEs
    data_assoc = { "ID antenne": list_antenne_sorted, "List ID UE": vector
    }
    df_assoc = pd.DataFrame(data=data_assoc)
    
    #Writing necessary files
    path = os.path.abspath("") + '\\' + pl_lab1_rand_eqn 
    path_inv = os.path.abspath("") + '\\' + assoc_lab1_rand_eqn 
    with open(path, 'w') as f:
        df_string = df.to_string(header=False, index=False)
        f.write(df_string)
    f.close()
    with open(path_inv, 'w') as f_inv:
        df_string_inv = df_assoc.to_string(header=False, index=False)
        f_inv.write(df_string_inv)
    f_inv.close()



### Fonction Main


In [48]:
def lab1_eqn() :
    print(my_variable)
    N_ue = 300
    N_antenne = 25
    coords_ue, coords_antennes = place_ue_and_antennas(N_antenne, N_ue)

    user_choose_name = input("Voulez-vous choisir le nom des fichiers à ecrire? ")   

    #pathloss random
    if user_choose_name == "oui" :
        pl_lab1_eqn = input("Quel nom de fichier pour pl_lab1_rand_eqn.txt ")
        assoc_lab1_eqn = input("Quel nom de fichier pour assoc_lab1_rand_eqn.txt ")
    else :
        pl_lab1_eqn = "pl_lab1_rand_eq8.txt"
        assoc_lab1_eqn = "assoc_lab1_rand_eq8.txt" 

    pathloss_computation = "random"
    write_file_assoc(pathloss_computation, pl_lab1_eqn, assoc_lab1_eqn, coords_ue, coords_antennes, N_ue, N_antenne)

    #pathloss proportional to distance
    if user_choose_name == "oui" :
        pl_lab1_eqn = input("Quel nom de fichier pour pl_lab1_dist_eqn.txt ")
        assoc_lab1_eqn = input("Quel nom de fichier pour assoc_lab1_dist_eqn.txt ")
    else :
        pl_lab1_eqn = "pl_lab1_dist_eq8.txt"
        assoc_lab1_eqn ="assoc_lab1_dist_eq8.txt"

    pathloss_computation = "distance"
    write_file_assoc(pathloss_computation, pl_lab1_eqn, assoc_lab1_eqn, coords_ue, coords_antennes, N_ue, N_antenne)
lab1_eqn()


NameError: name 'my_variable' is not defined